The following code is responsible for converting MIDI files into text notation.
# I found a way to use the original python library!
Reference here: https://github.com/jameswenzel/mydy/blob/master/src/Containers.py
https://github.com/jameswenzel/Fractal-Midi/blob/master/script.py
https://github.com/vishnubob/python-midi

In [11]:
from mydy import Events, FileIO, Containers
test=FileIO.read_midifile('file.mid')

track = Containers.Track()

print(test)

on = Events.NoteOnEvent(tick=50, velocity=50, pitch=30)
print(on)


mydy.Pattern(format=0, resolution=96, tracks=\
[mydy.Track(relative: True\
  [mydy.TrackNameEvent(tick=0, text='', data=[]),
   mydy.TimeSignatureEvent(tick=0, data=[4, 2, 36, 8]),
   mydy.TimeSignatureEvent(tick=0, data=[4, 2, 36, 8]),
   mydy.NoteOnEvent(tick=0, channel=0, data=[60, 100]),
   mydy.NoteOnEvent(tick=0, channel=0, data=[64, 100]),
   mydy.NoteOffEvent(tick=384, channel=0, data=[60, 64]),
   mydy.NoteOnEvent(tick=0, channel=0, data=[62, 100]),
   mydy.NoteOnEvent(tick=0, channel=0, data=[67, 100]),
   mydy.NoteOffEvent(tick=384, channel=0, data=[62, 64]),
   mydy.NoteOffEvent(tick=0, channel=0, data=[64, 64]),
   mydy.NoteOnEvent(tick=0, channel=0, data=[64, 100]),
   mydy.NoteOffEvent(tick=0, channel=0, data=[67, 64]),
   mydy.NoteOffEvent(tick=384, channel=0, data=[64, 64]),
   mydy.NoteOnEvent(tick=0, channel=0, data=[67, 100]),
   mydy.NoteOnEvent(tick=384, channel=0, data=[66, 100]),
   mydy.NoteOffEvent(tick=0, channel=0, data=[67, 64]),
   mydy.NoteOffEvent(tick=3